In [ ]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

#### Adapter Layer

In [ ]:
def csv_to_df(bucket, key, decoding='utf-8',sep=','):
    csv_obj = bucket.Object(key=key).get("Body").read().decode(decoding)
    data = StringIO(csv_obj)
    # read the csv data
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.get_value(), key=key)
    return True

def return_objects(bucket, args_date):
    min_date = datetime.strptime(args_date, src_format).date() - timedelta(days=1)
    objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
    return objects

#### Application Layer

In [ ]:
def extract(bucket, objects):
    df = pd.concat([csv_to_df(bucket, obj) for obj in objects], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(
        opening_price_eur=('opening_price', 'min'), 
        closing_price_eur=('closing_price', 'min'),
        minimum_price_eur=('MinPrice', 'min'),
        maximum_price_eur=('MaxPrice', 'max'),
        daily_trader_volume=('TradedVolume', 'sum')
    )
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    
    df['change_prev_closing_%'] = (
        df['closing_price_eur'] - df['prev_closing_price']
    ) / df['prev_closing_price'] * 100
    
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y-%m-%d_%H%M%S") + trg_format
    write_df_s3(bucket=bucket, df=df, key=key)
    return True

In [10]:
args_date = '2022-05-06'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
target_bucket = 'xetra-1234'
trg_key = 'xetra_daily_report'
trg_format = ".parquet"
columns = ["ISIN", "Date", "Time", "StartPrice", "MaxPrice", "MinPrice", "EndPrice", "TradeVolume"]

In [ ]:
# s3 resource
s3 = boto3.resource("s3")
bucket = s3.Bucket(src_bucket)


#### Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(
    opening_price_eur=('opening_price', 'min'), 
    closing_price_eur=('closing_price', 'min'),
    minimum_price_eur=('MinPrice', 'min'),
    maximum_price_eur=('MaxPrice', 'max'),
    daily_trader_volume=('TradedVolume', 'sum')
)

#### Percent change Previous Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all['change_prev_closing_%'] = (
    df_all['closing_price_eur'] - df_all['prev_closing_price']
) / df_all['prev_closing_price'] * 100

In [ ]:
df_all.drop(
    columns=['prev_closing_price'],
    inplace=True
)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all = df_all[df_all.Date >= args_date]

#### Write to S3

In [ ]:
# Target bucket instance
bucket_target = s3.Bucket(target_bucket)


#### Reading the uploaded file

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20230510_101629.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)